## Introduction à la Régression - Leçon 1

#### Mettre les choses en perspective

✅ Il existe de nombreux types de méthodes de régression, et le choix dépend de la réponse que vous cherchez. Si vous voulez prédire la taille probable d'une personne en fonction de son âge, vous utiliseriez la `régression linéaire`, car vous recherchez une **valeur numérique**. Si vous souhaitez déterminer si un type de cuisine doit être considéré comme végétalien ou non, vous cherchez une **catégorie** et utiliseriez donc la `régression logistique`. Vous en apprendrez davantage sur la régression logistique plus tard. Réfléchissez un peu aux questions que vous pourriez poser à partir des données, et à laquelle de ces méthodes elles conviendraient le mieux.

Dans cette section, vous travaillerez avec un [petit ensemble de données sur le diabète](https://www4.stat.ncsu.edu/~boos/var.select/diabetes.html). Imaginez que vous vouliez tester un traitement pour des patients diabétiques. Les modèles d'apprentissage automatique pourraient vous aider à déterminer quels patients répondraient mieux au traitement, en fonction de combinaisons de variables. Même un modèle de régression très basique, lorsqu'il est visualisé, pourrait révéler des informations sur des variables qui vous aideraient à organiser vos essais cliniques théoriques.

Cela dit, commençons cette tâche !

<p >
   <img src="../../images/encouRage.jpg"
   width="630"/>
   <figcaption>Illustration par @allison_horst</figcaption>

<!--![Illustration par \@allison_horst](../../../../../../translated_images/encouRage.e75d5fe0367fb9136b78104baf4e2032a7622bc42a2bc34c0ad36c294eeb83f5.fr.jpg)<br>Illustration par @allison_horst-->


## 1. Chargement de notre ensemble d'outils

Pour cette tâche, nous aurons besoin des packages suivants :

-   `tidyverse` : Le [tidyverse](https://www.tidyverse.org/) est une [collection de packages R](https://www.tidyverse.org/packages) conçue pour rendre la science des données plus rapide, plus facile et plus agréable !

-   `tidymodels` : Le framework [tidymodels](https://www.tidymodels.org/) est une [collection de packages](https://www.tidymodels.org/packages/) dédiée à la modélisation et à l'apprentissage automatique.

Vous pouvez les installer avec la commande suivante :

`install.packages(c("tidyverse", "tidymodels"))`

Le script ci-dessous vérifie si vous avez les packages nécessaires pour compléter ce module et les installe pour vous si certains sont manquants.


In [2]:
suppressWarnings(if(!require("pacman")) install.packages("pacman"))
pacman::p_load(tidyverse, tidymodels)

Loading required package: pacman



Maintenant, chargeons ces packages géniaux et rendons-les disponibles dans notre session R actuelle. (Ceci est juste pour illustration, `pacman::p_load()` l'a déjà fait pour vous)


In [ ]:
# load the core Tidyverse packages
library(tidyverse)

# load the core Tidymodels packages
library(tidymodels)


## 2. Le jeu de données sur le diabète

Dans cet exercice, nous allons mettre en pratique nos compétences en régression en réalisant des prédictions sur un jeu de données sur le diabète. Le [jeu de données sur le diabète](https://www4.stat.ncsu.edu/~boos/var.select/diabetes.rwrite1.txt) comprend `442 échantillons` de données relatives au diabète, avec 10 variables prédictives, notamment `l'âge`, `le sexe`, `l'indice de masse corporelle`, `la pression artérielle moyenne`, et `six mesures de sérum sanguin`, ainsi qu'une variable cible `y` : une mesure quantitative de la progression de la maladie un an après la ligne de base.

|Nombre d'observations|442|
|----------------------|:---|
|Nombre de prédicteurs|Les 10 premières colonnes sont des prédicteurs numériques|
|Cible/Variable cible|La colonne 11 est une mesure quantitative de la progression de la maladie un an après la ligne de base|
|Informations sur les prédicteurs|- âge en années
||- sexe
||- bmi indice de masse corporelle
||- bp pression artérielle moyenne
||- s1 tc, cholestérol sérique total
||- s2 ldl, lipoprotéines de basse densité
||- s3 hdl, lipoprotéines de haute densité
||- s4 tch, cholestérol total / HDL
||- s5 ltg, possiblement le logarithme du taux de triglycérides sériques
||- s6 glu, taux de sucre dans le sang|




> 🎓 Rappelez-vous, il s'agit d'un apprentissage supervisé, et nous avons besoin d'une cible nommée 'y'.

Avant de pouvoir manipuler des données avec R, vous devez importer les données dans la mémoire de R ou établir une connexion permettant à R d'accéder aux données à distance.

> Le package [readr](https://readr.tidyverse.org/), qui fait partie de Tidyverse, offre une manière rapide et conviviale de lire des données rectangulaires dans R.

Maintenant, chargeons le jeu de données sur le diabète disponible à cette URL source : <https://www4.stat.ncsu.edu/~boos/var.select/diabetes.html>

Nous effectuerons également une vérification de cohérence sur nos données en utilisant `glimpse()` et afficherons les 5 premières lignes avec `slice()`.

Avant d'aller plus loin, introduisons également un élément que vous rencontrerez souvent dans le code R 🥁🥁 : l'opérateur pipe `%>%`

L'opérateur pipe (`%>%`) permet d'exécuter des opérations dans une séquence logique en passant un objet à une fonction ou une expression d'appel. Vous pouvez considérer l'opérateur pipe comme signifiant "et ensuite" dans votre code.


In [ ]:
# Import the data set
diabetes <- read_table2(file = "https://www4.stat.ncsu.edu/~boos/var.select/diabetes.rwrite1.txt")


# Get a glimpse and dimensions of the data
glimpse(diabetes)


# Select the first 5 rows of the data
diabetes %>% 
  slice(1:5)

`glimpse()` nous montre que ces données contiennent 442 lignes et 11 colonnes, toutes de type de données `double`.

<br>

> glimpse() et slice() sont des fonctions de [`dplyr`](https://dplyr.tidyverse.org/). Dplyr, qui fait partie du Tidyverse, est une grammaire de manipulation de données offrant un ensemble cohérent de verbes pour résoudre les défis les plus courants en matière de manipulation de données.

<br>

Maintenant que nous avons les données, concentrons-nous sur une seule caractéristique (`bmi`) à cibler pour cet exercice. Cela nécessitera de sélectionner les colonnes souhaitées. Alors, comment faire cela ?

[`dplyr::select()`](https://dplyr.tidyverse.org/reference/select.html) nous permet de *sélectionner* (et éventuellement renommer) des colonnes dans un tableau de données.


In [ ]:
# Select predictor feature `bmi` and outcome `y`
diabetes_select <- diabetes %>% 
  select(c(bmi, y))

# Print the first 5 rows
diabetes_select %>% 
  slice(1:10)

## 3. Données d'entraînement et de test

Il est courant en apprentissage supervisé de *diviser* les données en deux sous-ensembles : un ensemble (généralement plus grand) pour entraîner le modèle, et un ensemble plus petit "mis de côté" pour évaluer les performances du modèle.

Maintenant que nous avons des données prêtes, nous pouvons voir si une machine peut aider à déterminer une séparation logique entre les nombres de ce jeu de données. Nous pouvons utiliser le package [rsample](https://tidymodels.github.io/rsample/), qui fait partie du framework Tidymodels, pour créer un objet contenant les informations sur *comment* diviser les données, puis deux autres fonctions de rsample pour extraire les ensembles d'entraînement et de test créés :


In [ ]:
set.seed(2056)
# Split 67% of the data for training and the rest for tesing
diabetes_split <- diabetes_select %>% 
  initial_split(prop = 0.67)

# Extract the resulting train and test sets
diabetes_train <- training(diabetes_split)
diabetes_test <- testing(diabetes_split)

# Print the first 3 rows of the training set
diabetes_train %>% 
  slice(1:10)

## 4. Entraîner un modèle de régression linéaire avec Tidymodels

Nous sommes maintenant prêts à entraîner notre modèle !

Dans Tidymodels, vous spécifiez les modèles en utilisant `parsnip()` en définissant trois concepts :

-   Le **type** de modèle distingue les modèles tels que la régression linéaire, la régression logistique, les modèles d'arbres de décision, etc.

-   Le **mode** du modèle inclut des options courantes comme la régression et la classification ; certains types de modèles prennent en charge l'un ou l'autre, tandis que d'autres n'ont qu'un seul mode.

-   Le **moteur** du modèle est l'outil de calcul qui sera utilisé pour ajuster le modèle. Souvent, il s'agit de packages R, tels que **`"lm"`** ou **`"ranger"`**.

Ces informations sur le modèle sont capturées dans une spécification de modèle, alors construisons-en une !


In [ ]:
# Build a linear model specification
lm_spec <- 
  # Type
  linear_reg() %>% 
  # Engine
  set_engine("lm") %>% 
  # Mode
  set_mode("regression")


# Print the model specification
lm_spec

Après qu'un modèle a été *spécifié*, il peut être `estimé` ou `entraîné` en utilisant la fonction [`fit()`](https://parsnip.tidymodels.org/reference/fit.html), généralement avec une formule et des données.

`y ~ .` signifie que nous allons ajuster `y` comme la quantité/variable cible prédite, expliquée par tous les prédicteurs/caractéristiques, c'est-à-dire `.` (dans ce cas, nous n'avons qu'un seul prédicteur : `bmi`).


In [ ]:
# Build a linear model specification
lm_spec <- linear_reg() %>% 
  set_engine("lm") %>%
  set_mode("regression")


# Train a linear regression model
lm_mod <- lm_spec %>% 
  fit(y ~ ., data = diabetes_train)

# Print the model
lm_mod

À partir des résultats du modèle, nous pouvons observer les coefficients appris lors de l'entraînement. Ils représentent les coefficients de la ligne de meilleure adéquation qui minimise l'erreur globale entre la variable réelle et la variable prédite.
<br>

## 5. Faire des prédictions sur le jeu de test

Maintenant que nous avons entraîné un modèle, nous pouvons l'utiliser pour prédire la progression de la maladie y pour le jeu de données de test en utilisant [parsnip::predict()](https://parsnip.tidymodels.org/reference/predict.model_fit.html). Cela servira à tracer la ligne entre les groupes de données.


In [ ]:
# Make predictions for the test set
predictions <- lm_mod %>% 
  predict(new_data = diabetes_test)

# Print out some of the predictions
predictions %>% 
  slice(1:5)

Youhou ! 💃🕺 Nous venons de former un modèle et de l'utiliser pour faire des prédictions !

Lorsqu'on effectue des prédictions, la convention de tidymodels est de toujours produire un tibble/data frame de résultats avec des noms de colonnes standardisés. Cela permet de combiner facilement les données originales et les prédictions dans un format utilisable pour des opérations ultérieures, comme la création de graphiques.

`dplyr::bind_cols()` lie efficacement plusieurs data frames par colonne.


In [ ]:
# Combine the predictions and the original test set
results <- diabetes_test %>% 
  bind_cols(predictions)


results %>% 
  slice(1:5)

## 6. Visualiser les résultats du modèle

Il est maintenant temps de voir cela visuellement 📈. Nous allons créer un nuage de points avec toutes les valeurs de `y` et `bmi` du jeu de test, puis utiliser les prédictions pour tracer une ligne à l'endroit le plus approprié, entre les regroupements de données du modèle.

R propose plusieurs systèmes pour créer des graphiques, mais `ggplot2` est l'un des plus élégants et des plus polyvalents. Cela permet de composer des graphiques en **combinant des composants indépendants**.


In [ ]:
# Set a theme for the plot
theme_set(theme_light())
# Create a scatter plot
results %>% 
  ggplot(aes(x = bmi)) +
  # Add a scatter plot
  geom_point(aes(y = y), size = 1.6) +
  # Add a line plot
  geom_line(aes(y = .pred), color = "blue", size = 1.5)

✅ Réfléchissez un peu à ce qui se passe ici. Une ligne droite traverse de nombreux petits points de données, mais que fait-elle exactement ? Pouvez-vous voir comment cette ligne pourrait vous permettre de prédire où un nouveau point de données, encore jamais vu, devrait se situer par rapport à l'axe y du graphique ? Essayez de mettre en mots l'utilité pratique de ce modèle.

Félicitations, vous avez construit votre premier modèle de régression linéaire, créé une prédiction avec celui-ci, et l'avez affichée dans un graphique !



---

**Avertissement** :  
Ce document a été traduit à l'aide du service de traduction automatique [Co-op Translator](https://github.com/Azure/co-op-translator). Bien que nous nous efforcions d'assurer l'exactitude, veuillez noter que les traductions automatisées peuvent contenir des erreurs ou des inexactitudes. Le document original dans sa langue d'origine doit être considéré comme la source faisant autorité. Pour des informations critiques, il est recommandé de faire appel à une traduction humaine professionnelle. Nous déclinons toute responsabilité en cas de malentendus ou d'interprétations erronées résultant de l'utilisation de cette traduction.
